# Stochastic Signal Detection

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly as py
from plotly import express as px
from sqlalchemy import create_engine

py.offline.init_notebook_mode(connected=True)


## Visualization

We use SQLite and `pandas` to load the simulation results.

In [3]:
db = 'simulation.sqlite'
table = 'stochastic_signal_detection'
output = Path('visualization')
output.mkdir(exist_ok=True, parents=True)
sql = create_engine(f'sqlite:///{db}')
df = pd.read_sql_table(table, sql, index_col='id')


The table groups various variables of the simulation:

In [4]:
df.describe()


,timestamp,seed,rows,ratio,rank,beta,mass_scale,mass_scale_bottom,mass_scale_top,mu_0,...,T,nmax_chi_0,max_chi_0,argmax_chi_0,nmax_chi_1,max_chi_1,argmax_chi_1,min_frac,nmin_frac,argmin_frac
count,3878,3878.000000,3878.0,3.878000e+03,3878.0,3878.000000,3878.000000,3878.000000,3878.000000,3878.000000,...,3878.000000,3878.000000,3.878000e+03,3878.000000,3878.000000,3.878000e+03,3878.000000,3878.000000,3878.000000,3878.000000
mean,2023-09-16 09:34:56.963898880,12412.527849,10000.0,8.000000e-01,2500.0,0.322157,0.070005,0.047500,0.100091,0.271132,...,0.771132,745.516503,-1.969150e+10,0.284442,997.798865,2.180912e+298,0.309247,-21.531230,77.665807,0.195816
min,2023-09-15 22:12:43,1151.000000,10000.0,8.000000e-01,2500.0,0.000000,0.000000,0.000000,0.028786,-0.500000,...,0.000000,5.000000,-5.156030e+13,0.089418,936.000000,-9.491134e+300,0.169833,-9385.102827,0.000000,0.000000
25%,2023-09-16 03:40:39.249999872,5039.000000,10000.0,8.000000e-01,2500.0,0.210000,0.033882,0.009178,0.064029,-0.150000,...,0.350000,486.250000,-3.340468e+01,0.231282,997.000000,-4.159299e+37,0.252937,-3.986698,2.000000,0.096636
50%,2023-09-16 09:12:24,9706.000000,10000.0,8.000000e-01,2500.0,0.315000,0.067024,0.042356,0.097057,0.250000,...,0.750000,1000.000000,-2.363043e+00,0.278781,999.000000,4.608034e+00,0.311540,-0.464180,7.000000,0.229139
75%,2023-09-16 15:41:20,22766.000000,10000.0,8.000000e-01,2500.0,0.420000,0.100749,0.075829,0.130773,0.650000,...,1.150000,1001.000000,4.885464e+00,0.343617,1000.000000,2.212063e+29,0.361712,0.350589,27.000000,0.293042
max,2023-09-16 21:31:15,26843.000000,10000.0,8.000000e-01,2500.0,0.525000,0.130356,0.105496,0.160445,1.050000,...,1.550000,1001.000000,1.729176e+08,0.400632,1001.000000,3.169008e+301,0.400632,0.997094,1001.000000,0.400632
std,NaN,9176.865770,0.0,1.110366e-16,0.0,0.136839,0.039168,0.035848,0.039247,0.465570,...,0.465570,387.109295,8.629248e+11,0.069258,4.173906,inf,0.065806,240.589103,223.445778,0.114679


In particular, the final columns represent the "measure of criticality" of the simulation:

- `nmax_chi_0` is the position of the maximum of the derivative of the potential in the list of values of $k$ used for the simulation for $\chi = 0$, i.e.: $n = \mathrm{pos}\max\left|\mathcal{U}'[0]\right|$, where $n \in [0, 500]$;
- `argmax_chi_0` is the position at the maximum of the derivative for $\chi = 0$, i.e.: $\overline{k} = \arg\max\left|\mathcal{U}'[0]\right|$;
- `max_chi_0` is the value at the maximum of the derivative for $\chi = 0$, i.e.: $\mathcal{U}'[0](\overline{k})$;
- `nmax_chi_1` is the position of the maximum of the derivative of the potential in the list of values of $k$ used for the simulation for $\chi = 1$, i.e.: $n = \mathrm{pos}\max\left|\mathcal{U}'[1]\right|$, where $n \in [0, 500]$;
- `argmax_chi_1` is the position at the maximum of the derivative for $\chi = 1$, i.e.: $\overline{k} = \arg\max\left|\mathcal{U}'[1]\right|$;
- `max_chi_1` is the value at the maximum of the derivative for $\chi = 1$, i.e.: $\mathcal{U}'[1](\overline{k})$.

We first add the _symmetric_ logarithm of the quantities measured in the simulation:

In [5]:
df['log_max_chi_0'] = df['max_chi_0'].apply(lambda x: np.log(x)
                                            if x > 0 else -np.log(-x))
df['abs_log_max_chi_0'] = df['max_chi_0'].apply(lambda x: np.log(np.abs(x)))
df['abs_max_chi_0'] = df['max_chi_0'].apply(lambda x: np.abs(x))
df['log_max_chi_1'] = df['max_chi_1'].apply(lambda x: np.log(x)
                                            if x > 0 else -np.log(-x))
df['abs_log_max_chi_1'] = df['max_chi_1'].apply(lambda x: np.log(np.abs(x)))
df['abs_max_chi_1'] = df['max_chi_1'].apply(lambda x: np.abs(x))


## Scan in the parameters

First of all, we group by seed the various simulations:

In [6]:
df = df.groupby(['T', 'beta'
                 ]).mean().reset_index().drop(columns=['timestamp', 'seed'])
df.describe()


,T,beta,rows,ratio,rank,mass_scale,mass_scale_bottom,mass_scale_top,mu_0,mu_1,...,argmax_chi_1,min_frac,nmin_frac,argmin_frac,log_max_chi_0,abs_log_max_chi_0,abs_max_chi_0,log_max_chi_1,abs_log_max_chi_1,abs_max_chi_1
count,512.000000,512.00000,512.0,5.120000e+02,512.0,512.000000,512.000000,512.000000,512.000000,512.000000,...,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,5.120000e+02,512.000000,512.000000,5.120000e+02
mean,0.775000,0.26250,10000.0,8.000000e-01,2500.0,0.054781,0.035978,0.084773,0.275000,0.775000,...,0.280642,-17.059477,60.359821,0.148637,-2.848211,7.419866,2.670995e+10,-35.375479,275.562989,1.080750e+299
std,0.462106,0.16150,0.0,1.101498e-16,0.0,0.043604,0.037285,0.043772,0.462106,0.462106,...,0.076997,75.213867,157.707869,0.129699,8.542344,6.830277,3.845835e+11,254.321652,247.184536,inf
min,0.000000,0.00000,10000.0,8.000000e-01,2500.0,0.000000,0.000000,0.029450,-0.500000,0.000000,...,0.171551,-1007.842207,0.000000,0.000000,-27.954499,-0.271453,7.622718e-01,-618.745761,-0.271453,7.622718e-01
25%,0.387500,0.13125,10000.0,8.000000e-01,2500.0,0.013339,0.000000,0.043449,-0.112500,0.387500,...,0.208346,-6.352948,2.000000,0.000643,-6.791655,1.484117,4.754280e+00,-34.389612,43.612264,3.144100e+23
50%,0.775000,0.26250,10000.0,8.000000e-01,2500.0,0.049068,0.024326,0.079196,0.275000,0.775000,...,0.281168,-0.970326,6.100000,0.163325,-0.602815,3.379587,5.487632e+01,-1.375442,151.630682,1.676266e+73
75%,1.162500,0.39375,10000.0,8.000000e-01,2500.0,0.091539,0.066814,0.121659,0.662500,1.162500,...,0.348599,0.276159,26.150000,0.274395,0.620637,14.591396,3.703421e+06,17.944803,548.233778,1.554094e+252
max,1.550000,0.52500,10000.0,8.000000e-01,2500.0,0.129509,0.104752,0.159687,1.050000,1.550000,...,0.399631,0.993352,1000.200000,0.384810,18.808467,27.954499,7.706747e+12,604.598859,664.377984,1.584504e+301


We then list the independent temperature values explored in the dataset:

In [7]:
Ts = list(df['T'].unique())


We then explore the various parameters of the simulation:

In [8]:
group = df.groupby('T')


We are finally ready to visualize:

In [9]:
N = 0

data = group.get_group(Ts[N])
fig = px.scatter(data,
                 x='beta',
                 y='abs_max_chi_1',
                 title=f'T = {Ts[N]}',
                 log_y=True)
fig.update_layout(xaxis=dict(title=r'$\beta$'),
                  yaxis=dict(title=r'$|\max \mathcal{U}\'[1]|$',
                             tickformat='.0e'),
                  template='ggplot2')
fig.update_traces(
    hovertemplate=r'$\beta = $%{x}<br>$\max \mathcal{U}\'[1] = $%{y}',
    marker=dict(symbol='0', size=10, color='red'))
fig.show()


In [10]:
M = 0

data = group.nth(M)
fig = px.scatter_3d(data,
                    x='mu_0',
                    y='mu_1',
                    z='mu_2',
                    color='abs_log_max_chi_1',
                    custom_data=['abs_log_max_chi_1', 'T'],
                    color_continuous_scale='turbo',
                    title=fr'$\beta = ${data["beta"].mean():.2f}')
fig.update_layout(
    scene={
        'xaxis': dict(title=r'$\mu_0$'),
        'yaxis': dict(title=r'$\mu_1$'),
        'zaxis': dict(title=r'$\mu_2$'),
    },
    coloraxis={'colorbar': dict(title='$\ln|\max \mathcal{U}\'[1]|$')},
    template='ggplot2')
fig.update_traces(
    hovertemplate=
    r'$\mu_0 = $%{x}<br>$\mu_1 = $%{y}<br>$\mu_2 = $%{z}<br>$\ln|\max \mathcal{U}\'[0]| = $%{customdata[0]}<br>$T = $%{customdata[1]}'
)
fig.update
fig.show()


In [24]:
fig = px.scatter_3d(df,
                    x='beta',
                    y='T',
                    z='abs_log_max_chi_1',
                    color='abs_log_max_chi_0',
                    color_continuous_scale='turbo',
                    custom_data=['abs_log_max_chi_0'])
fig.update_layout(
    scene={
        'xaxis': dict(title=r'$\beta$', range=[-0.01, 0.6]),
        'yaxis': dict(title='T'),
        'zaxis': dict(title=r'$\ln|\max \mathcal{U}\'[0]|$'),
    },
    coloraxis={'colorbar': dict(title='$\ln|\max \mathcal{U}\'[0]|$')},
    template='ggplot2')
fig.update_traces(
    hovertemplate=
    r'$\beta = $%{x}<br>$T = $%{y}<br>$\ln|\max \mathcal{U}\'[1]| = $%{z}<br>$\ln|\max \mathcal{U}\'[0]| = $%{customdata[0]}',
    marker=dict(size=5))
fig.update
fig.show()
